## Loading the Youtube Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
# Read the JSON file
with open('../Data/watch-history.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.json_normalize(data)

## Exploratory Data Analysis (EDA)

In [3]:
df

,header,title,titleUrl,subtitles,time,products,activityControls,description,details
0,YouTube,Watched 10 PROJECTS TO ADD TO YOUR DATA PORTFOLIO,https://www.youtube.com/watch?v=wObV_hwu2QM,"[{'name': 'Data With Mo', 'url': 'https://www....",2023-11-19T13:20:50.835Z,[YouTube],[YouTube watch history],NaN,NaN
1,YouTube,Watched END TO END DATA ANALYST PORTFOLIO PROJ...,https://www.youtube.com/watch?v=nl9eZl1IOKI,"[{'name': 'Data With Mo', 'url': 'https://www....",2023-11-19T13:18:34.016Z,[YouTube],[YouTube watch history],NaN,NaN
2,YouTube,Watched Cueshé - Ulan,https://www.youtube.com/watch?v=HP12yvlVNs8,"[{'name': 'CuesheVEVO', 'url': 'https://www.yo...",2023-11-19T13:16:36.666Z,[YouTube],[YouTube watch history],NaN,NaN
3,YouTube,Watched Coldplay - Viva La Vida (Official Video),https://www.youtube.com/watch?v=dvgZkm1xWPE,"[{'name': 'Coldplay', 'url': 'https://www.yout...",2023-11-19T13:12:31.731Z,[YouTube],[YouTube watch history],NaN,NaN
4,YouTube,Watched Post Malone - Circles,https://www.youtube.com/watch?v=wXhTHyIgQ_U,"[{'name': 'PostMaloneVEVO', 'url': 'https://ww...",2023-11-19T13:08:44.198Z,[YouTube],[YouTube watch history],NaN,NaN
...,...,...,...,...,...,...,...,...,...
43095,YouTube,Watched https://www.youtube.com/watch?v=yRL7M9...,https://www.youtube.com/watch?v=yRL7M9pw3Rc,NaN,2023-07-17T03:59:05.456Z,[YouTube],[YouTube watch history],NaN,NaN
43096,YouTube,Watched https://www.youtube.com/watch?v=ugaSmU...,https://www.youtube.com/watch?v=ugaSmUCR9Sc,NaN,2023-07-17T03:59:04.515Z,[YouTube],[YouTube watch history],NaN,[{'name': 'From Google Ads'}]
43097,YouTube,Watched https://www.youtube.com/watch?v=rarmC_...,https://www.youtube.com/watch?v=rarmC_ONJz4,NaN,2023-07-17T03:58:04.552Z,[YouTube],[YouTube watch history],NaN,NaN
43098,YouTube,Watched https://www.youtube.com/watch?v=LxYfSo...,https://www.youtube.com/watch?v=LxYfSoRLuz8,NaN,2023-07-17T03:58:02.843Z,[YouTube],[YouTube watch history],NaN,NaN


In [4]:
df.shape

(43100, 9)

In [5]:
df.columns

Index(['header', 'title', 'titleUrl', 'subtitles', 'time', 'products',
       'activityControls', 'description', 'details'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43100 entries, 0 to 43099
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   header            43100 non-null  object
 1   title             43100 non-null  object
 2   titleUrl          43015 non-null  object
 3   subtitles         36310 non-null  object
 4   time              43100 non-null  object
 5   products          43100 non-null  object
 6   activityControls  43100 non-null  object
 7   description       3741 non-null   object
 8   details           5814 non-null   object
dtypes: object(9)
memory usage: 3.0+ MB


In [7]:
df.isnull().sum()

header                  0
title                   0
titleUrl               85
subtitles            6790
time                    0
products                0
activityControls        0
description         39359
details             37286
dtype: int64

In [8]:
df['header'].nunique

<bound method IndexOpsMixin.nunique of 0        YouTube
1        YouTube
2        YouTube
3        YouTube
4        YouTube
          ...   
43095    YouTube
43096    YouTube
43097    YouTube
43098    YouTube
43099    YouTube
Name: header, Length: 43100, dtype: object>

### Pre-processing

#### Drop some columns

In [9]:
columns_to_drop = ['header','subtitles', 'activityControls', 'products']
df.drop(columns=columns_to_drop, inplace=True)

#### Remove the "Watched" word in the title column

In [10]:
df['title'] = df['title'].str.replace('^Watched\s', '', regex=True)

#### Reformat the details columns to isolate the "From Google Ads"

In [11]:
def extract_name(details):
    if isinstance(details, list) and len(details) > 0:
        details_dict = details[0]
        if 'name' in details_dict:
            return details_dict['name']
    return np.nan

df['details'] = df['details'].apply(extract_name)

#### Create a Duration column based on the watch time

The calculation for the duration presented a challenge for me. The original logic I used was the basic subtract the time difference between each video. However, I discovered this logic was flawed since if I just simply subtracted via the time differences but I did not take into account the time I was offline/sleeping/doing-something-else outside Youtube so therefore the ending watch hours resulted in the whole timeframe from July to November essentially. I thought of many things to filter out that "Time Space". The solution I ultimately choosing is to drop anything longer than 5 hours and then continue the by row calculation. Yep it's a shortcut but it's a much better approximation than my previous method.

In [12]:
# Convert 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Calculate duration between consecutive rows and create a 'duration' column
df['duration'] = df['time'].diff().fillna(pd.Timedelta(seconds=0))

# Subtract duration from a full day and extract the time part
df['duration'] = pd.to_timedelta('1 day') - df['duration']
df['duration'] = df['duration'].dt.components['hours'].astype(str).str.zfill(2) + ':' + \
                 df['duration'].dt.components['minutes'].astype(str).str.zfill(2) + ':' + \
                 df['duration'].dt.components['seconds'].astype(str).str.zfill(2)

In [13]:
df

,title,titleUrl,time,description,details,duration
0,10 PROJECTS TO ADD TO YOUR DATA PORTFOLIO,https://www.youtube.com/watch?v=wObV_hwu2QM,2023-11-19 13:20:50.835000+00:00,NaN,NaN,00:00:00
1,END TO END DATA ANALYST PORTFOLIO PROJECT | Fr...,https://www.youtube.com/watch?v=nl9eZl1IOKI,2023-11-19 13:18:34.016000+00:00,NaN,NaN,00:02:16
2,Cueshé - Ulan,https://www.youtube.com/watch?v=HP12yvlVNs8,2023-11-19 13:16:36.666000+00:00,NaN,NaN,00:01:57
3,Coldplay - Viva La Vida (Official Video),https://www.youtube.com/watch?v=dvgZkm1xWPE,2023-11-19 13:12:31.731000+00:00,NaN,NaN,00:04:04
4,Post Malone - Circles,https://www.youtube.com/watch?v=wXhTHyIgQ_U,2023-11-19 13:08:44.198000+00:00,NaN,NaN,00:03:47
...,...,...,...,...,...,...
43095,https://www.youtube.com/watch?v=yRL7M9pw3Rc,https://www.youtube.com/watch?v=yRL7M9pw3Rc,2023-07-17 03:59:05.456000+00:00,NaN,NaN,00:00:52
43096,https://www.youtube.com/watch?v=ugaSmUCR9Sc,https://www.youtube.com/watch?v=ugaSmUCR9Sc,2023-07-17 03:59:04.515000+00:00,NaN,From Google Ads,00:00:00
43097,https://www.youtube.com/watch?v=rarmC_ONJz4,https://www.youtube.com/watch?v=rarmC_ONJz4,2023-07-17 03:58:04.552000+00:00,NaN,NaN,00:00:59
43098,https://www.youtube.com/watch?v=LxYfSoRLuz8,https://www.youtube.com/watch?v=LxYfSoRLuz8,2023-07-17 03:58:02.843000+00:00,NaN,NaN,00:00:01


In [14]:
df['time'] = pd.to_datetime(df['time'], errors='coerce')
df = df.sort_values('time')

# Duration calc
df['duration'] = (df['time'] - df['time'].shift()).fillna(pd.Timedelta(seconds=0)) / pd.Timedelta(hours=1)

# 5 hour duration filter
df = df[df['duration'] <= 5]

df.reset_index(drop=True, inplace=True)

# formatting
df['duration'] = pd.to_timedelta(df['duration'], unit='h')
df['duration'] = df['duration'].dt.components['hours'].astype(str).str.zfill(2) + ':' + \
                 df['duration'].dt.components['minutes'].astype(str).str.zfill(2) + ':' + \
                 df['duration'].dt.components['seconds'].astype(str).str.zfill(2)

In [15]:
df

,title,titleUrl,time,description,details,duration
0,https://www.youtube.com/watch?v=Yj5oXpuK6O4,https://www.youtube.com/watch?v=Yj5oXpuK6O4,2023-07-17 03:57:09.268000+00:00,NaN,NaN,00:00:00
1,https://www.youtube.com/watch?v=LxYfSoRLuz8,https://www.youtube.com/watch?v=LxYfSoRLuz8,2023-07-17 03:58:02.843000+00:00,NaN,NaN,00:00:53
2,https://www.youtube.com/watch?v=rarmC_ONJz4,https://www.youtube.com/watch?v=rarmC_ONJz4,2023-07-17 03:58:04.552000+00:00,NaN,NaN,00:00:01
3,https://www.youtube.com/watch?v=ugaSmUCR9Sc,https://www.youtube.com/watch?v=ugaSmUCR9Sc,2023-07-17 03:59:04.515000+00:00,NaN,From Google Ads,00:00:59
4,https://www.youtube.com/watch?v=yRL7M9pw3Rc,https://www.youtube.com/watch?v=yRL7M9pw3Rc,2023-07-17 03:59:05.456000+00:00,NaN,NaN,00:00:00
...,...,...,...,...,...,...
43006,I Found a Samsung Galaxy S22 Underwater (THEN...),https://www.youtube.com/watch?v=BCw9zTV15BU,NaT,NaN,NaN,00:00:00
43007,When the door says pull but it’s push day 😔,https://www.youtube.com/watch?v=Gs_2SHaHFrM,NaT,NaN,NaN,00:00:00
43008,수삼국 vt154c,https://www.youtube.com/watch?v=Gfu7OmQkyPg,NaT,Watched at 18:54\nWatched at 16:11,From Google Ads,00:00:00
43009,Best ad forever 🔥🤣The end😳#genshinimpact #gens...,https://www.youtube.com/watch?v=a-p1JHXsjmE,NaT,NaN,NaN,00:00:00


In [16]:
df['duration'].max()

'04:59:43'

Condition met

In [17]:
# Convert the 'duration' column to timedelta format
df['duration'] = pd.to_timedelta(df['duration'])

# Calculate the total watch time in seconds
total_watch_time_seconds2 = df['duration'].dt.total_seconds().sum()

# Convert total watch time to a more readable format (hours, minutes, seconds)
hours2 = int(total_watch_time_seconds2 // 3600)
minutes2 = int((total_watch_time_seconds2 % 3600) // 60)
seconds2 = int(total_watch_time_seconds2 % 60)

print(f"Total watch time: {hours2} hours, {minutes2} minutes, {seconds2} seconds")

Total watch time: 2316 hours, 15 minutes, 19 seconds


In [18]:
df['duration'] = pd.to_timedelta(df['duration'], unit='h')
df['duration'] = df['duration'].dt.components['hours'].astype(str).str.zfill(2) + ':' + \
                 df['duration'].dt.components['minutes'].astype(str).str.zfill(2) + ':' + \
                 df['duration'].dt.components['seconds'].astype(str).str.zfill(2)